In [1]:
!pip install datasets
!pip install -q pytorch-lightning nltk
!pip install sentencepiece
!pip uninstall -y ninja && pip install ninja
!pip install wandb
!pip install OmegaConf


Found existing installation: ninja 1.11.1.1
Uninstalling ninja-1.11.1.1:
  Successfully uninstalled ninja-1.11.1.1
  Obtaining dependency information for ninja from https://files.pythonhosted.org/packages/6d/92/8d7aebd4430ab5ff65df2bfee6d5745f95c004284db2d8ca76dcbfd9de47/ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.6 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 964.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=15d86444f5b8ef4927f9a4cb4ff80809ff1eecb9229c59d737ea4cc9fedf357c
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [ ]:
!nvcc --version

In [ ]:

# Check PyTorch version and GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"CUDNN version: {torch.backends.cudnn.version() if torch.cuda.is_available() else 'N/A'}")

In [ ]:
!ninja --version

In [ ]:
!echo $?

In [2]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.7 MB/s eta 0:00:0000:010:010m
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.1 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.0-cp310-cp310-linux_x86_64.whl size=121788330 sha256=180f415c86683f9ff8ecbcee4333405107c550cc7d0d16830eb62f89438e46f3
  Stored in directory: /root/.cache/pip/wheels/9e/c3/22/a576eb5627fb2c30dc4679a33d67d34d922d6dbeb24a9119b2
Successfully built flash-attn


In [3]:
!pip install /kaggle/input/flash-a/flash-attention-1.0.9/csrc/fused_dense_lib

Processing /kaggle/input/flash-a/flash-attention-1.0.9/csrc/fused_dense_lib
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      /opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:398: UserWarning: There are no g++ version bounds defined for CUDA version 11.8
        warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
      error: could not create '/kaggle/input/flash-a/flash-attention-1.0.9/csrc/fused_dense_lib/build': Read-only file system
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fused-dense-lib
  Running setup.py clean for fused-dense-lib
Failed to build fused-dense-lib
ERROR: Could not build wheels for fused-dense-lib, which is required to install pyproject.toml-based proje

In [ ]:
# !pip install /content/flash_attention/flash-attention-main/csrc/xentropy
# !pip install /content/flash_attention/flash-attention-main/csrc/rotary
# !pip install /content/flash_attention/flash-attention-main/csrc/layer_norm

In [2]:
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from flash_attn.models.gpt import GPTLMHeadModel

seqlen = 512
hidden_dim = 2048
nheads = 16
n_layer = 24
rotary_emb_fraction = 0.5
config = GPT2Config(vocab_size=50257, n_positions=seqlen, n_embd=hidden_dim,
                    n_layer=n_layer, n_head=nheads,
                    scale_attn_by_inverse_layer_idx=True,
                    rotary_emb_fraction=rotary_emb_fraction,
                    use_flash_attn=True, fused_mlp=True,
                    fused_bias_fc=True, fused_dropout_add_ln=True,
                    pad_vocab_size_multiple=8)
model = GPTLMHeadModel(config)

ImportError: fused_dense is not installed

In [ ]:
print(model)

In [ ]:
import torch
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader, Dataset
from transformers import AutoTokenizer
from datasets import load_dataset
from pytorch_lightning import LightningDataModule

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

import pytorch_lightning as pl

import wandb

In [ ]:
from datasets import load_dataset

data =load_dataset("wikimedia/wikisource", "20231201.en")

In [ ]:
print(data)

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import DataLoader, Dataset, random_split
tokenizer =  GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
class Wikidata(Dataset):
    def __init__(self, dataset, max_length=512,tokenizer=tokenizer,data_collator=data_collator):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.data_collator = data_collator
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Get text from the dataset
        text = self.dataset[idx]["text"]  # Access the list using idx and then get "text" from the dictionary

        # Tokenize the text using the GPT-2 tokenizer
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=self.max_length,
            padding="max_length"
        )

        # For GPT-2 language modeling, the target is the same as the input shifted by one position
        labels = inputs["input_ids"].clone()
        labels[:, :-1] = inputs["input_ids"][:, 1:]
        labels[:, -1] = self.tokenizer.eos_token_id  # Set the last token to the EOS token

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "labels": labels.squeeze()
        }


In [ ]:
class WikiDataModule(pl.LightningDataModule):
    def __init__(self, dataset, batch_size=1, max_length=512, val_split=0.1, test_split=0.1, num_workers=2):
        super().__init__()
        self.dataset = dataset['train']
        self.batch_size = batch_size
        self.max_length = max_length
        self.val_split = val_split
        self.test_split = test_split
        self.num_workers = num_workers

    def setup(self, stage=None):
        train_size = int(len(self.dataset) * (1 - self.val_split - self.test_split))
        val_size = int(len(self.dataset) * self.val_split)
        test_size = len(self.dataset) - train_size - val_size
        self.train_dataset, self.val_dataset, self.test_dataset = random_split(self.dataset, [train_size, val_size, test_size])
        self.train_dataset = Wikidata(self.train_dataset, self.max_length)
        self.val_dataset = Wikidata(self.val_dataset, self.max_length)
        self.test_dataset = Wikidata(self.test_dataset, self.max_length)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=self.num_workers, collate_fn=self.train_dataset.data_collator)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers, collate_fn=self.val_dataset.data_collator)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers, collate_fn=self.test_dataset.data_collator)


In [ ]:
batch_size = 1
max_length = 512
num_workers = 4

wikidatamodule = WikiDataModule(data, batch_size=batch_size, max_length=max_length, num_workers=num_workers)
wikidatamodule.setup()
train_dataloader = wikidatamodule.train_dataloader()
val_dataloader = wikidatamodule.val_dataloader()

In [ ]:
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from flash_attn.models.gpt import GPTLMHeadModel
import pytorch_lightning as pl

seqlen = 2048
hidden_dim = 2048
nheads = 16
n_layer = 24
rotary_emb_fraction = 0.5
config = GPT2Config(vocab_size=50257, n_positions=seqlen, n_embd=hidden_dim,
                    n_layer=n_layer, n_head=nheads,
                    scale_attn_by_inverse_layer_idx=True,
                    rotary_emb_fraction=rotary_emb_fraction,
                    use_flash_attn=True, fused_mlp=True,
                    fused_bias_fc=True, fused_dropout_add_ln=True,
                    pad_vocab_size_multiple=8)
model = GPTLMHeadModel(config)




In [ ]:
import torch
import pytorch_lightning as pl
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from flash_attn.models.gpt import GPTLMHeadModel

class LightningGPTModel(pl.LightningModule):
    def __init__(self, config):
        super(LightningGPTModel, self).__init__()
        self.model = GPTLMHeadModel(config)


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
        return outputs.loss

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        outputs = self(input_ids, attention_mask, labels=labels)
        loss = outputs

        return loss


    def configure_optimizers(self):
        # Define your optimizer and scheduler (if any) here
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return optimizer

# Your existing code to set up the model configuration
seqlen = 512
hidden_dim = 2048
nheads = 16
n_layer = 24
rotary_emb_fraction = 0.5
config = GPT2Config(vocab_size=50257, n_positions=seqlen, n_embd=hidden_dim,
                    n_layer=n_layer, n_head=nheads,
                    scale_attn_by_inverse_layer_idx=True,
                    rotary_emb_fraction=rotary_emb_fraction,
                    use_flash_attn=True, fused_mlp=True,
                    fused_bias_fc=True, fused_dropout_add_ln=True,
                    pad_vocab_size_multiple=8)

# Create an instance of the LightningGPTModel
lightning_model = LightningGPTModel(config)

In [ ]:

class LightningGPTModel(pl.LightningModule):
    def __init__(self, config):
        super(LightningGPTModel, self).__init__()
        self.model = GPTLMHeadModel(config)

    def forward(self, input_ids, attention_mask=None):
        return self.model(input_ids)

    def training_step(self, batch, batch_idx):
        # The 'labels' tensor is automatically passed to this method by PyTorch Lightning
        inputs = {"input_ids": batch["input_ids"]}

        # Handle attention_mask separately if your model requires it
        if "attention_mask" in batch:
            inputs["attention_mask"] = batch["attention_mask"]

        outputs = self.model(**inputs, labels=batch["labels"])
        loss = outputs.loss
        return loss









In [ ]:
print(lightning_model)

In [ ]:
data_module = WikiDataModule(dataset=data, batch_size=1, max_length=512, val_split=0.1, test_split=0.1, num_workers=2)
data_module.setup()  # Call setup to initialize train, val, and test datasets

trainer_config = {
    'max_epochs': 5,

    # 'gpus': 1,
    # 'accumulate_grad_batches': 1,
    # # Add other trainer configurations as needed
}

# Step 4: Instantiate Trainer
trainer = pl.Trainer(**trainer_config,detect_anomaly=True)

# Step 5: Train the Model
trainer.fit(lightning_model, datamodule=data_module)

# Step 6: (Optional) Test the Model
trainer.test(datamodule=data_module)